# Sentiment analysis với dữ liệu foody sử dụng fasttext

In [2]:
import fasttext
import os,sys  
import pandas as pd
import numpy as np
import fasttext
import re
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.filterwarnings('ignore')
print(sys.version)

3.5.2 (default, Sep 14 2017, 22:51:06) 
[GCC 5.4.0 20160609]


## Sử dụng library fastext của python

In [5]:
#load data
data = pd.read_pickle('./foody review.pkl')
data.head()

,user_link,user_name,brand_name,brand_link,review_content,score
0,https://www.foody.vn/thanh-vien/taiyo,Sunny,El Sol - Meat & Wine - Võ Thị Sáu,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,Quán steak hiếm hoi mà mình cực kì ưng ý từ lâ...,8.6
1,https://www.foody.vn/thanh-vien/viola_0309_tv,Nguyên Khánh,Busan Korean Food - Món Hàn Quốc - Đinh Tiên H...,https://www.foody.vn/ho-chi-minh/busan-korean-...,Vị trí dễ tìm. Giữ xe rất nhiệt tình. Dắt xe v...,7.4
2,https://www.foody.vn/thanh-vien/tuhao2509,Tú Hảo,TocoToco Bubble Tea - Cộng Hòa,https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,"Địa điểm quán dễ tìm. Không gian cũng rộng, có...",6.4
3,https://www.foody.vn/thanh-vien/zip_zilip,Vy Nhật,Le Castella Viet Nam - Bánh Bông Lan Đài Loan ...,https://www.foody.vn/ho-chi-minh/le-castella-v...,Thấy bánh này đang hot rần rần mình cũng gọi t...,7.2
4,https://www.foody.vn/thanh-vien/tien97,Tien97,Tabletop - Boardgame & Coffee,https://www.foody.vn/ho-chi-minh/tabletop-boar...,Mỗi lần nghĩ tới boardgame thì sẽ nghĩ tới vô ...,10.0


In [6]:
def string_clean(s):
    s_lower = s.lower()
    s_clean = " ".join(re.findall("[0-9a-zaáàảãạâấầẩẫậăắằẳẵặeéèẻẽẹêếềểễệiíìỉĩịoóòỏõọôốồổỗộơớờởỡợuúùủũụưứừửữựyýỳỷỹỵđ]+", s_lower))
    return(s_clean)

In [7]:
%%time
reviews = data[['review_content', 'score']]
#Để cho đơn giản, giả định điểm số không nhỏ hơn 5 là tốt còn ngược lại là tệ
reviews['score'] = reviews['score'].apply(lambda x: '__label__good' if x >= 5 else '__label__bad')
reviews['review_content'] = reviews['review_content'].apply(lambda x: string_clean(x))

CPU times: user 7.5 s, sys: 72 ms, total: 7.57 s
Wall time: 7.57 s


In [8]:
review_list = reviews['score'] + ' ' +  reviews['review_content']

In [9]:
##chia tập train và test
x_train, x_test = train_test_split(review_list, test_size=0.2, random_state=42, shuffle=True)

In [10]:
#ghi dữ liệu
train_dir = './review.train'
x_train.to_csv(train_dir, header=None, index=False )

test_dir = './review.test'
x_test.to_csv(test_dir, header=None, index=False )

In [11]:
%%time
#Train a classifier
output_file = './review_model'
classifier = fasttext.supervised(train_dir, output_file)

CPU times: user 19 s, sys: 440 ms, total: 19.4 s
Wall time: 6.39 s


In [12]:
# Evaluate classifier
result = classifier.test(test_dir)
print('Precision:', result.precision)
print('Recall:', result.recall)

Precision: 0.9587108613373093
Recall: 0.9587108613373093


## Thử dự đoán
Thử dự đoán với một số review mới trong ngày của foody tại https://www.foody.vn/ha-noi#/reviews

In [15]:
def model_predict(string):
    sentence = [string_clean(string)]
    labels = classifier.predict_proba(sentence)
    classs, probs = labels[0][0]
    print("Label: %s; certainty: %f" %(classs, probs))

In [16]:
reviews = """
Quán của "ông chủ hay mặc áo màu hường" kaka cũng thấy thú vị khi có quen biết tí chút về ông chủ quán. 
Kể ra chú em nó mở hàng được 1 thời gian rồi mà hôm nay mới đặt về ăn được. 
Hôm nay đặt em nó món bún ốc chuối đậu. 
Điểm cộng của quán là chủ quán rất lắng nghe ý kiến đóng góp sản phẩm từ khách hàng. 
1 suất hết 35k nhưng khá đầy đủ và ốc khá nhiều. 
Điểm chưa được cộng là về nước dùng của ốc chuối đậu chưa được sánh( chưa chuẩn vị ốc chuối đậu) nhìn giống canh chuối đậu hơn. 
Kaka chỉ cần chỉnh xíu xíu nữa là món ăn sẽ hài hòa hơn nhé " Ông chủ hay mặc áo màu hường"
"""

model_predict(reviews)

Label: good; certainty: 0.970703


In [17]:
reviews = """
Tàm tạm
Bố của ngọt, mẹ của ngọt ,70% mà rất rất ngọt. Nên ko thấy vị trà đâu 😭😭
"""

model_predict(reviews)

Label: good; certainty: 0.966797


In [18]:
reviews = """
Sẽ không bao giờ quay lại.
Lần đầu tiên trong cuộc đời mình sẽ review quán ăn vì trải nghiệm của mình quá đau thương. 

Lúc đầu khi đến quán, sau khi được giải thích là suất buffet 179k/ng thì thịt unlimited còn nấm sẽ chỉ ra 1 lần, 
bọn mình quyết định gọi 5 suất này, hồn nhiên suy nghĩ là gọi 5 suất thì sẽ có 5 đĩa nấm đi kèm. 
Nhưng không, bạn nhân viên mang ra MỘT đĩa nấm SIÊU BÉ và giải thích rằng nấm "ra 1 lần" tức là ra 1 đĩa cho 1 bàn. 
Mình có chụp ảnh lại đĩa nấm. 
Đĩa đó mà các bạn serve cho 5 người lớn một cách hồn nhiên như vậy thì mình cũng thấy phục các bạn ?!?!!!

Đến thời điểm này, bọn mình quyết định đổi sang suất buffet 279k/ng. 
Để nói về đồ ăn, mình ko thấy dở nhưng cũng ko thấy ngon. 
Mình chỉ có ấn tượng là các bạn thiết kế bàn ngồi không rộng rãi lắm nhưng các loại đĩa đựng thức ăn và quyển menu rất to (trong khi thức ăn trong đĩa thì ít - mình thấy đây là một sự phí phạm không gian). 
Khách ngồi sắp đĩa ra bàn và mở đc quyển menu cũng hết hơi 😭 Nước lẩu mình ăn tứ xuyên thì quá mỡ nên mình ko cảm nhận được bất cứ vị gì ở bên nước còn lại nữa 😂 sau khi ăn xong thì đến ngày hôm sau mình vẫn còn ngấy... 
và điều tồi tệ nhất là trong nhóm 5 người thì có 2 người bị đau bụng. 
"""

model_predict(reviews)

Label: bad; certainty: 0.605469


In [19]:
reviews = """
Trải nghiệm sâu sắc
Hnay mình đặt 2 suất cơm tại đây. 
Đặt qua delivery now từ 11h40. 
Hẹn 12h giao hàng sau đó bị rời giờ đến 12h30 và cuối cùng là 13h30 mới nhận được 2 suất cơm này. 
Cơm canh lạnh ngắt. 
1 suất thì bị thiếu rau. 2 cốc canh cho được 1 cái thìa. 😡😡😡.  
1 sao cũng không cho nổi
"""

model_predict(reviews)

Label: bad; certainty: 0.855469


# Sử dụng trực tiếp fasttext trên ubuntu

## Train model

In [40]:
%%time

FT_HOME = './fasttext'
folder_dir = './fasttext/'
output = 'review_model_1'
!{FT_HOME} supervised -input {folder_dir + 'review.train'} -output {output} -wordNgrams 2

Read 15M words
Number of words:  40143
Number of labels: 2
Progress: 60.4%  words/sec/thread: 1663317  lr: 0.039646  loss: 0.132529  eta: 0h0m h-14m 1617214  lr: 0.099782  loss: 0.687401  eta: 0h0m 0.6%  words/sec/thread: 1651440  lr: 0.099359  loss: 0.459615  eta: 0h0m %  words/sec/thread: 1656694  lr: 0.098984  loss: 0.378140  eta: 0h0m   loss: 0.307669  eta: 0h0m %  words/sec/thread: 1665274  lr: 0.097353  loss: 0.243176  eta: 0h0m 3.0%  words/sec/thread: 1669804  lr: 0.096968  loss: 0.239554  eta: 0h0m %  words/sec/thread: 1671262  lr: 0.096569  loss: 0.236040  eta: 0h0m %  words/sec/thread: 1681351  lr: 0.095774  loss: 0.234725  eta: 0h0m 4.7%  words/sec/thread: 1686516  lr: 0.095289  loss: 0.231202  eta: 0h0m   lr: 0.094449  loss: 0.222730  eta: 0h0m 6.8%  words/sec/thread: 1694478  lr: 0.093185  loss: 0.207121  eta: 0h0m 7.6%  words/sec/thread: 1693306  lr: 0.092378  loss: 0.196325  eta: 0h0m 0.091580  loss: 0.191252  eta: 0h0m %  words/sec/thread: 1694536  lr: 0.090712  loss: 0

## Test model

In [41]:
!{FT_HOME} test {folder_dir + output + '.bin'} {folder_dir + 'review.test'}

N	34876
P@1	0.96
R@1	0.96
Number of examples: 34876


## Thử dự đoán

In [49]:
def model_predict_1(string):
    f = open(folder_dir + 'review_text.txt', 'w')    
    s_clean = string_clean(string)
    f.write(s_clean)
    f.close()
    !{FT_HOME} predict-prob {folder_dir + output + '.bin'} {folder_dir + 'review_text.txt'} 

In [50]:
reviews = """
Quán của "ông chủ hay mặc áo màu hường" kaka cũng thấy thú vị khi có quen biết tí chút về ông chủ quán. 
Kể ra chú em nó mở hàng được 1 thời gian rồi mà hôm nay mới đặt về ăn được. 
Hôm nay đặt em nó món bún ốc chuối đậu. 
Điểm cộng của quán là chủ quán rất lắng nghe ý kiến đóng góp sản phẩm từ khách hàng. 
1 suất hết 35k nhưng khá đầy đủ và ốc khá nhiều. 
Điểm chưa được cộng là về nước dùng của ốc chuối đậu chưa được sánh( chưa chuẩn vị ốc chuối đậu) nhìn giống canh chuối đậu hơn. 
Kaka chỉ cần chỉnh xíu xíu nữa là món ăn sẽ hài hòa hơn nhé " Ông chủ hay mặc áo màu hường"
"""

model_predict_1(reviews)

__label__good 0.96875


In [51]:
reviews = """
Trải nghiệm sâu sắc
Hnay mình đặt 2 suất cơm tại đây. 
Đặt qua delivery now từ 11h40. 
Hẹn 12h giao hàng sau đó bị rời giờ đến 12h30 và cuối cùng là 13h30 mới nhận được 2 suất cơm này. 
Cơm canh lạnh ngắt. 
1 suất thì bị thiếu rau. 2 cốc canh cho được 1 cái thìa. 😡😡😡.  
1 sao cũng không cho nổi
"""

model_predict_1(reviews)

__label__bad 0.947266
